In [31]:
import numpy as np
import random
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler


#Store Data Variables
import json
with open('feature_data.json', 'r') as f:
 features = json.load(f)

from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
filelist = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
camId = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()

In [32]:
train_idxs

array([    1,     2,     3, ..., 14094, 14095, 14096], dtype=uint16)

In [33]:
query_idxs

array([   22,    27,    49, ..., 14043, 14059, 14063], dtype=uint16)

In [34]:
labels

array([   1,    1,    1, ..., 1467, 1467, 1467], dtype=uint16)

In [35]:
gallery_idxs

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [36]:
filelist

array([array(['1_001_1_01.png'], dtype='<U14'),
       array(['1_001_1_02.png'], dtype='<U14'),
       array(['1_001_1_03.png'], dtype='<U14'), ...,
       array(['5_049_2_08.png'], dtype='<U14'),
       array(['5_049_2_09.png'], dtype='<U14'),
       array(['5_049_2_10.png'], dtype='<U14')], dtype=object)

In [37]:
camId

array([1, 1, 1, ..., 2, 2, 2], dtype=uint8)

In [38]:
#grid searc cv against k, distance metric and transformation/normalization - use only query and gallery for this part 

In [39]:
#scaler = StandardScaler()
print(np.array(features))
#features = scaler.fit_transform(features)
X = np.array(features)
print(X)
y = np.array(labels)
filelist = np.array(filelist)
camId = np.array(camId)

[[0.13791114 1.12505555 0.05404324 ... 0.10747871 0.04081609 0.68009406]
 [0.02507781 0.9057585  0.00346441 ... 0.16763815 0.14764351 0.2700713 ]
 [0.09923808 1.09122825 0.01889733 ... 0.07981343 0.04958951 0.37923682]
 ...
 [0.50913167 2.1067946  1.12224829 ... 0.78767842 1.68007588 0.00259321]
 [0.44684452 1.87411916 1.51910186 ... 1.02090526 1.58616257 0.10876646]
 [0.46024311 2.31024432 1.20531154 ... 0.62753201 1.22394812 0.        ]]
[[0.13791114 1.12505555 0.05404324 ... 0.10747871 0.04081609 0.68009406]
 [0.02507781 0.9057585  0.00346441 ... 0.16763815 0.14764351 0.2700713 ]
 [0.09923808 1.09122825 0.01889733 ... 0.07981343 0.04958951 0.37923682]
 ...
 [0.50913167 2.1067946  1.12224829 ... 0.78767842 1.68007588 0.00259321]
 [0.44684452 1.87411916 1.51910186 ... 1.02090526 1.58616257 0.10876646]
 [0.46024311 2.31024432 1.20531154 ... 0.62753201 1.22394812 0.        ]]


In [40]:
mask_train = np.array(train_idxs).ravel()
mask_query = np.array(query_idxs).ravel()
mask_gallery = np.array(gallery_idxs).ravel()

mask_train = np.subtract(mask_train, 1)
mask_query = np.subtract(mask_query, 1)
mask_gallery = np.subtract(mask_gallery, 1)


X_train, X_query, X_gallery = X[mask_train, :], X[mask_query, :], X[mask_gallery, :]
y_train, y_query, y_gallery = y[mask_train], y[mask_query], y[mask_gallery]
filelist_train, filelist_query, filelist_gallery = filelist[mask_train], filelist[mask_query], filelist[mask_gallery]
camId_train, camId_query, camId_gallery = camId[mask_train], camId[mask_query], camId[mask_gallery]


In [41]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(4096,), activation='logistic', max_iter=10, alpha=1e-4,
                    solver='adam', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)

In [42]:
for Xtnn, ytnn in zip(X_train, y_train):
    X_train_pairs = []
    y_train_pair_lables = []
    dists = []
    dists_indexes = []
    for i,(Xtnn2, ytnn2) in enumerate (zip(X_train, y_train)):
        if (ytnn != ytnn2):
            dist = distance.euclidean(Xtnn, Xtnn2)
            dists.append(dist)
            dists_indexes.append(i)
    
    dists = np.array(dists)
    dists_indexes = np.array(dists_indexes)
    _indexes = np.argsort(dists)

    # Sorted distances and labels
    dists, dists_indexes = dists[_indexes], dists_indexes[_indexes] 
        
    for Xtnn2, ytnn2, camId_t2 in zip(X_train, y_train, camId_train):
        if (ytnn == ytnn2):
            for i in range (30):
                Xtnn3 = X_train[dists_indexes[i]]
                ytnn3 = y_train[dists_indexes[i]]                 

                dist2 = 0
                if(ytnn == ytnn3):
                    dist3 = 0
                else:
                    dist3 = 1

                Xconcat = np.concatenate((Xtnn,Xtnn2,Xtnn3), axis = None)
                X_train_pairs.append(Xconcat)
                y_train_pair_lables.append((dist2,dist3))   

                Xconcat = np.concatenate((Xtnn,Xtnn3,Xtnn2), axis = None)
                X_train_pairs.append(Xconcat)
                y_train_pair_lables.append((dist3,dist2)) 

    y_train_pair_lables = np.array(y_train_pair_lables)
    X_train_pairs = np.array(X_train_pairs)
    if (X_train_pairs.shape[0] != 0):
            mlp.partial_fit(X_train_pairs, y_train_pair_lables)      

Iteration 1, loss = 8858.06953764
Iteration 2, loss = 9276.47819570
Iteration 3, loss = 2078.22433063
Iteration 4, loss = 5457.73149540
Iteration 5, loss = 820.33844118
Iteration 6, loss = 2875.69355098
Iteration 7, loss = 457.04492418
Iteration 8, loss = 1533.52783994
Iteration 9, loss = 266.76012693
Iteration 10, loss = 815.73180795
Iteration 11, loss = 144.44623068
Iteration 12, loss = 429.08574692
Iteration 13, loss = 78.64075661
Iteration 14, loss = 218.60864693
Iteration 15, loss = 61.34727467
Iteration 16, loss = 86.95341572
Iteration 17, loss = 53.91543289
Iteration 18, loss = 30.08372415
Iteration 19, loss = 42.84304954
Iteration 20, loss = 12.20681956
Iteration 21, loss = 18.83831336
Iteration 22, loss = 7.15558008
Iteration 23, loss = 3.31723146
Iteration 24, loss = 3.27888654
Iteration 25, loss = 3.55931898
Iteration 26, loss = 2.60582774
Iteration 27, loss = 2.40897770
Iteration 28, loss = 2.57959775
Iteration 29, loss = 1.93172111
Iteration 30, loss = 1.71985990
Iteration

Iteration 160, loss = 0.78620907
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 161, loss = 0.75054631
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 162, loss = 0.73695248
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 163, loss = 0.74201129
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 164, loss = 0.71022796
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 165, loss = 0.69769852
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 166, loss = 0.69797271
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 167, loss = 0.68314106
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteratio

Iteration 277, loss = 2.50380987
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 278, loss = 2.58374282
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 279, loss = 1.70745895
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 280, loss = 1.59086259
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 281, loss = 1.26926823
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 282, loss = 1.38339731
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 283, loss = 1.30854676
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 284, loss = 1.30008881
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteratio

Iteration 343, loss = 1.14031441
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 344, loss = 1.15736308
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 345, loss = 0.98700840
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 346, loss = 0.86663650
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 347, loss = 0.79641055
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 348, loss = 0.71500194
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 349, loss = 0.73458754
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 350, loss = 0.67629314
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteratio

Iteration 409, loss = 24.19449233
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 410, loss = 11.82517417
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 411, loss = 8.79990619
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 412, loss = 7.99429447
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 413, loss = 7.26684470
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 414, loss = 9.30784572
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 415, loss = 7.30829496
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 416, loss = 5.61404470
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iterat

...

Iteration 7087, loss = 17.34095005
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7088, loss = 16.30193137
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7089, loss = 15.49592450
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7090, loss = 14.88727088
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7091, loss = 14.44098451
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7092, loss = 14.13203367
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7093, loss = 13.92845824
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7094, loss = 15.33976755
Training loss did not improve more than tol=0.000100 for two consecutive epochs. St

Iteration 7153, loss = 31.33851953
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7154, loss = 68.36628469
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7155, loss = 211.42335686
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7156, loss = 167.32657454
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7157, loss = 122.59904430
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7158, loss = 91.00377653
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7159, loss = 68.80806015
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7160, loss = 53.14634620
Training loss did not improve more than tol=0.000100 for two consecutive epochs.

Iteration 7218, loss = 16.67075277
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7219, loss = 16.04335711
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7220, loss = 15.27922837
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7221, loss = 28.92874225
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7222, loss = 32.79052531
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7223, loss = 36.92517232
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7224, loss = 36.61008994
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7225, loss = 64.50188674
Training loss did not improve more than tol=0.000100 for two consecutive epochs. St

Iteration 7283, loss = 20.22384195
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7284, loss = 23.14145213
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7285, loss = 21.22249894
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7286, loss = 19.91049742
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7287, loss = 18.62818515
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7288, loss = 17.71292163
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7289, loss = 18.97502248
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7290, loss = 24.40157131
Training loss did not improve more than tol=0.000100 for two consecutive epochs. St

Iteration 7349, loss = 22.15515771
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7350, loss = 22.02410604
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7351, loss = 22.31288862
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7352, loss = 28.83605664
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7353, loss = 30.96614244
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7354, loss = 29.36412953
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7355, loss = 26.11955455
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7356, loss = 26.40030322
Training loss did not improve more than tol=0.000100 for two consecutive epochs. St

In [43]:
def get_acc_score(y_valid, y_q, tot_label_occur):
    recall = 0
    true_positives = 0
    
    k = 0
    
    max_rank = 30
    
    rank_A = np.zeros(max_rank)
    AP_arr = np.zeros(11)
    
    while (recall < 1) or (k < max_rank):
        if (y_valid[k] == y_q):
            
            true_positives = true_positives + 1
            recall = true_positives/tot_label_occur
            precision = true_positives/(k+1)
            
            AP_arr[round((recall-0.05)*10)] = precision
            
            for n in range (k, max_rank):
                rank_A[n] = 1
            
        k = k+1
        
    max_precision = 0
    for i in range(10, -1, -1):
        max_precision = max(max_precision, AP_arr[i])
        AP_arr[i] = max_precision
    
    AP_ = AP_arr.sum()/11
    
    return AP_, rank_A

In [44]:
from scipy.spatial import distance

def evaluate_metric(X_query, camId_query, y_query, X_gallery, camId_gallery, y_gallery, metric = mlp, parameters = None):

    rank_accuracies = []
    AP = []

    # Break condition for testing
    #q = 0

    for query, camId_q, y_q in zip(X_query, camId_query, y_query):
        q_g_dists = []
        y_valid = []
        for gallery, camId_g, y_g  in zip(X_gallery, camId_gallery, y_gallery):
            if ((camId_q == camId_g) and (y_q == y_g)):
                continue
            else:
                if metric == mlp:
                    dist = mlp.predict(np.concatenate((query, gallery, np.zeros(2048))).reshape((1,6144)))[0][0]
                elif metric == 'minkowski':
                    dist = distance.minkowski(query, gallery, parameters)
                else:
                    raise NameError('Specified metric not supported')           
                q_g_dists.append(dist)
                y_valid.append(y_g)
    
        tot_label_occur = y_valid.count(y_q)
    
        q_g_dists = np.array(q_g_dists)
        y_valid = np.array(y_valid)
        
    
        _indexes = np.argsort(q_g_dists)
    
        # Sorted distances and labels
        q_g_dists, y_valid = q_g_dists[_indexes], y_valid[_indexes]
    
        AP_, rank_A = get_acc_score(y_valid, y_q, tot_label_occur)
    
        AP.append(AP_)
        
        rank_accuracies.append(rank_A) 
    
        #if q  > 5:
        #    break
        #q = q+1

    rank_accuracies = np.array(rank_accuracies)

    total = rank_accuracies.shape[0]
    rank_accuracies = rank_accuracies.sum(axis = 0)
    rank_accuracies = np.divide(rank_accuracies, total)

    i = 0
    print ('Accuracies by Rank:')
    while i < rank_accuracies.shape[0]:
        print('Rank ', i+1, ' = %.2f%%' % (rank_accuracies[i] * 100), '\t',
              'Rank ', i+2, ' = %.2f%%' % (rank_accuracies[i+1] * 100), '\t',
              'Rank ', i+3, ' = %.2f%%' % (rank_accuracies[i+2] * 100), '\t',
              'Rank ', i+4, ' = %.2f%%' % (rank_accuracies[i+3] * 100), '\t',
              'Rank ', i+5, ' = %.2f%%' % (rank_accuracies[i+4] * 100))
        i = i+5

    AP = np.array(AP)

    mAP = AP.sum()/AP.shape[0]
    print('mAP = %.2f%%' % (mAP * 100))
    
    return rank_accuracies, mAP

In [45]:
rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric = mlp,
                                       parameters = None)

KeyboardInterrupt: 